In [13]:
import torch
from tqdm.notebook import tqdm, trange
import gpytorch
import numpy as np

In [14]:
# seed everything
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the data

Here, we load data from the tabular benchmark suite https://github.com/LeoGrin/tabular-benchmark 

ids from https://arxiv.org/pdf/2207.08815.pdf, appendix A.1.2

In [15]:
import openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

dataset_idx = 3
type_ids = {
    44132,  # 0 cpu-act
    44133,  # 1 pol
    44134,  # 2 elevators
    44135,  # 3 isolet
    44136,  # 4 wine-quality
    44137,  # 5 Ailerons
    44138,  # 6 houses
    44139,  # 7 house-16H
    44141,  # 8 Brazilian-houses
    44142,  # 9 Bike-Sharing-Demand
    44144,  # 10 house-sales
    44145,  # 11 sulfur
    44147,  # 12 MiamiHousing2016
    44148,  # 13 superconduct
    44025,  # 14 california
    44026,  # 15 fifa
}
dataset_id = list(type_ids)[dataset_idx]
dataset = openml.datasets.get_dataset(dataset_id)
# dataset_name = dataset.name
X, y, _, _ = dataset.get_data(dataset_format='dataframe', target=dataset.default_target_attribute)
X, y = X.values, y.values.astype(np.float32)

# split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)

# to torch and device
X_train = torch.tensor(X_train, dtype=torch.float32, device=device)
y_train = torch.tensor(y_train, dtype=torch.float32, device=device)
X_test = torch.tensor(X_test, dtype=torch.float32, device=device)
y_test = torch.tensor(y_test, dtype=torch.float32, device=device)

# Normalize the data

In [16]:
class TorchStandardScaler:
    def __init__(self):
        self.mean_ = None
        self.std_ = None
    def fit(self, X):
        self.mean_ = X.mean(dim=0)
        self.std_ = X.std(dim=0)
    def transform(self, X):
        return (X - self.mean_) / self.std_
    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)
    def inverse_transform(self, X):
        return X * self.std_ + self.mean_
    def inverse_transform_std(self, X):
        return X * self.std_

Xscaler = TorchStandardScaler()
yscaler = TorchStandardScaler()

X_train = Xscaler.fit_transform(X_train)
X_test = Xscaler.transform(X_test)

y_train = yscaler.fit_transform(y_train)
y_test = yscaler.transform(y_test)

# Define the model


In [17]:
from models_gp import ExpMahalanobisDistanceKernel
from recursive_feature_machine import LaplaceRFM


# define the GP model
class GPLaplaceMahalanobis(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, weight_matrix):
        super(GPLaplaceMahalanobis, self).__init__(train_x, train_y, likelihood)
        self.weight_matrix = weight_matrix.to(train_x)

        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            ExpMahalanobisDistanceKernel(
                weight_matrix=self.weight_matrix,
                squared=False,
                ard_num_dims=1,
            )
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


rfm_reg = 1e-3
rfm_ridge = 1e-7
rfm_diag = False

# define the RFM model
model_rfm = LaplaceRFM(
    bandwidth=1,
    device=device,
    reg=rfm_reg,
    ridge=rfm_ridge,
    verbose=False,
    diag=rfm_diag,
    centering=True,
)

# Train the RFM model

In [18]:
%%time

model_rfm.fit(
    (X_train, y_train), (X_test, y_test),
    loader=False,iters=5,
    classif=False,
)

weight_matrix = model_rfm.M
del model_rfm

# define the GP model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model_gp = GPLaplaceMahalanobis(X_train, y_train, likelihood, weight_matrix)

# move to device
likelihood = likelihood.to(device)
model_gp = model_gp.to(device)

CPU times: user 224 ms, sys: 56.1 ms, total: 280 ms
Wall time: 280 ms


# Train the GP model

In [19]:
lr = 0.05
n_epochs = 250


def train_gp():
    model_gp.train()
    likelihood.train()

    # optimizer
    optimizer = torch.optim.Adam(model_gp.parameters(), lr=0.1)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=n_epochs, eta_min=1e-7)

    # loss, the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model_gp)

    with trange(n_epochs, desc='Training GP') as pbar:
        for _ in pbar:
            # Zero gradients from previous iteration
            optimizer.zero_grad(set_to_none=True)
            # Output from model
            output = model_gp(X_train)
            # Calc loss and backprop gradients
            loss = -mll(output, y_train)
            loss.backward()
            optimizer.step()
            scheduler.step()

            pbar.set_postfix(loss=loss.item())


# train the GP model
%time train_gp()

Training GP:   0%|          | 0/250 [00:00<?, ?it/s]

CPU times: user 8.22 s, sys: 1.46 s, total: 9.67 s
Wall time: 11.2 s


# Evaluate the model

In [20]:
model_gp.eval()
likelihood.eval()

with torch.no_grad(), gpytorch.settings.fast_pred_var():
    preds = likelihood(model_gp(X_test))

# unnormalize
y_pred_mean = yscaler.inverse_transform(preds.mean)
y_pred_std = yscaler.inverse_transform_std(preds.stddev)
y_test = yscaler.inverse_transform(y_test)

# calculate the RMSE
rmse = torch.sqrt(torch.mean((y_pred_mean - y_test) ** 2, dim=-1))
print(f'RMSE: {rmse}')
# calculate the NLL
covariance_matrix = torch.diag_embed(y_pred_std ** 2)
mvn = torch.distributions.MultivariateNormal(y_pred_mean, covariance_matrix)
nll = -mvn.log_prob(y_test) / y_test.shape[0]
print(f'NLL: {nll}')


RMSE: 2.6848392486572266
NLL: 2.382169723510742
